# Compute Frame Similarity Using SSIM(Structural Similarity Index)

In [30]:
from skimage.measure import compare_ssim
import argparse
import imutils
import cv2
import os
import pandas as pd

# Link dos steps

https://www.pyimagesearch.com/2017/06/19/image-difference-with-opencv-and-python/

# load the two input images

In [9]:
imagemA = cv2.imread("output_scenes_frames/6334458-360p_487k_h264/cena-1/frame_2.0.jpg")

In [14]:
imagemB = cv2.imread("output_scenes_frames/6334458-360p_487k_h264/cena-2/frame_100.0.jpg")

# convert the images to grayscale

In [15]:
grayA = cv2.cvtColor(imagemA, cv2.COLOR_BGR2GRAY)

In [16]:
grayB = cv2.cvtColor(imagemB, cv2.COLOR_BGR2GRAY)

# Compute the Structural Similarity Index (SSIM) between the two images, ensuring that the difference image is returned

In [17]:
(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))

SSIM: 0.49143576728317107


# Função para avaliar em qual momento a abertura aparece no vídeo

In [20]:
def calculate_similarity(path_frame1, path_frame2):
    imagemA = cv2.imread(path_frame1)
    imagemB = cv2.imread(path_frame2)
    
    #converter para escala de cinza
    grayA = cv2.cvtColor(imagemA, cv2.COLOR_BGR2GRAY)
    grayB = cv2.cvtColor(imagemB, cv2.COLOR_BGR2GRAY)
    
    #cálculo de similaridade
    (score, _) = compare_ssim(grayA, grayB, full=True)
    #diff = (diff * 255).astype("uint8")
    
    return score

In [27]:
def load_dataset(path_dataset):
    
    datasets = []
    
    #listar arquivos do dataset de abertura
    for ds_file in os.listdir(path_dataset):
        
        # criar path do arquivo
        file_ds_frame_path = "{}/{}".format(path_dataset,ds_file)
        
        # ler a imagem
        img_file = cv2.imread(file_ds_frame_path)
        
        # converter para cinza
        gray_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        
        # adiciona a imagem na lista
        datasets.append(gray_file)
    
    return datasets 

In [55]:
SIMLARITY_THRESHOLD = 0.8

def frames_sao_similares(frame1, frame2):
    (score, _) = compare_ssim(frame1, frame2, full=True)
    return score >= SIMLARITY_THRESHOLD

def cena_é_parte_da_abertura(frames_cena, datasets_abertura):
    for frame_cena in frames_cena:
        for frame_abr in datasets_abertura:
            if frames_sao_similares(frame_cena, frame_abr):
                return True
    return False

PATH_CSV_CENAS = 'scenes_tmp/6334458-360p_487k_h264-Scenes.csv'
PATH_CENAS_VIDEO = "output_scenes_frames/6334458-360p_487k_h264/"
PATH_DATASET_ABERTURA = "datasets_abertura/"

def encontrar_cenas_da_abertura(path_cenas_video, path_dataset_abertura):
    #dataset_abertura = load_dataset(path_dataset_abertura)
    cenas_abertura = []
    for cena_path in path_cenas_video:
        cena = load_dataset(path_video_scene)
        if cena_é_parte_da_abertura(cena, dataset_abertura):
            cenas_abertura.append(cena)
    return cenas_abertura

def main():
    df_metadados_cenas = pd.read_csv(PATH_CSV_CENAS, skiprows=1)
    dataset_abertura = load_dataset(PATH_DATASET_ABERTURA)
    
    lista_cenas = []
    
    for index, row in df_metadados_cenas.iterrows():
        if index > 4: 
            break
        print("Processando cena {}".format(row["Scene Number"]))
        scene_number = row["Scene Number"]
        path_cena = "{}/cena-{}".format(PATH_CENAS_VIDEO, str(scene_number))
        
        frames_cena = load_dataset(path_cena)
        if cena_é_parte_da_abertura(frames_cena, dataset_abertura):
            print("A cena faz parte da abertura !")
            lista_cenas.append(scene_number)
        else:
            print("A cena não faz parte !")
    return lista_cenas

In [ ]:
main()